# Imports

In [10]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
# import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]
import ray

# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# .describe(include='all', datetime_is_numeric=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'
# pd.set_option('precision', 2)
# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

from data_model import arrow_dataset, fsspec_storage, db
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, stacked, meta, daily_stats
import utilities

config = {
    'meta': {
        'symbol': 'GORO',
        'start_date': '2020-11-10',
        'end_date': '2020-11-16',
        'config_id': 'renko_v1',
    },
    'filter': {
        'mad_value_winlen': 22,
        'mad_deviation_winlen': 1111,
        'mad_k': 17,
        'jma_winlen': 7,
        'jma_power': 2,
    },
    'sampler': {
        'renko_return': 'price_jma_return',
        'renko_size': 0.1,  # for simple runs
        'renko_reveral_multiple': 2,
        'renko_range_frac': 22,
        'renko_range_min_pct_value': 0.03,  # % of symbol value enforced as min renko size
        'max_duration_td': dt.timedelta(minutes=33),
        'min_duration_td': dt.timedelta(seconds=33),
        'min_tick_count': 33,
        'add_label': True,
        'reward_ratios': list(np.arange(2, 11, 0.5)),
    }
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
db.get_symbol_storage_used('GLD', '/data/trades')

{'MB': 1031.3, 'GB': 1.03}

In [11]:
db.get_symbol_storage_used(symbol='GLD', prefix='data/trades')

{'MB': 1031.3, 'GB': 1.03}

In [15]:
db.write_sdf_to_local(df, 'BOB', '2022-01-01', '/test/test')

/Users/bobcolner/QuantClarity/DATA_ROOT//test/test/symbol=BOB/date=2022-01-01/


In [17]:
df = db.read_sdf('GLD', '2020-01-02', 'data/trades')

# Sample tick dates

In [ ]:
ray.init(dashboard_port=1111, ignore_reinit_error=True)

# ray.shutdown()

In [ ]:
meta.tick_sampler_workflow(config)


In [ ]:
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bds, fill_col='price_vwap')

stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked_df[['price_high','price_low','price_vwap']].plot()

In [ ]:
# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)

stats_df

stats_df.describe()